In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense , Flatten , Conv2D , MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
(x_train , y_train) , (x_test , y_test) = tf.keras.datasets.mnist.load_data() # load mnist dataset

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
print("x_train Shape: {}, y_train Shape: {}\n".format(x_train.shape , y_train.shape))
print("x_test Shape: {}, y_test Shape: {}\n".format(x_test.shape , y_test.shape))

x_train Shape: (60000, 28, 28), y_train Shape: (60000,)

x_test Shape: (10000, 28, 28), y_test Shape: (10000,)



In [ ]:
#plotting first 9 images and label
print("First 9 Samples:")
plt.figure(figsize = (7,7))
for i in range(9):
  plt.subplot(3,3,i+1)
  plt.imshow(x_train[i] , cmap = 'gray') # show the image
  label = y_train[i]
  plt.title(str(label)) # label of the image as image title
  plt.axis('off')

plt.show()

In [ ]:
# Pad dataset to make 48 x 48
x_train = np.pad(x_train, ((0,0),(10,10),(10,10)))
x_test = np.pad(x_test, ((0,0),(10,10),(10,10)))

# Convert image into 3 channels
x_train = np.stack((x_train,)*3, axis=-1)
x_test = np.stack((x_test,)*3, axis=-1)

x_train = x_train.astype('float32') / 255 # normalizing in range 0 to 1
x_test = x_test.astype('float32') / 255 # normalizing in range 0 to 1

y_train = to_categorical(y_train , 10) # one hot encoding of y_train
y_test = to_categorical(y_test , 10) # one hot encoding of y_test

print("Train data shape: ",x_train.shape ,"\nTest data shape:", x_test.shape)

Train data shape:  (60000, 48, 48, 3) 
Test data shape: (10000, 48, 48, 3)


In [ ]:
datagen = ImageDataGenerator(
    rotation_range=10,  # rotate the image up to 10 degrees
    width_shift_range=0.1,  # shift the image horizontally up to 10% of the width
    height_shift_range=0.1,  # shift the image vertically up to 10% of the height
    shear_range=0.2,  # shear the image up to 20%
    zoom_range=0.2,  # zoom the image up to 20%
    fill_mode='nearest'  # fill any empty pixels with the nearest pixel value
)

In [ ]:
# Generate augmented images on the fly during training
datagen.fit(x_train)
train_generator = datagen.flow(x_train, y_train, batch_size=32)

In [ ]:
#import VGG16 model from keras
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
model = tf.keras.Sequential() # define sequential model

model.add(VGG16(include_top=False,pooling='avg',weights='imagenet',input_shape = (48,48,3))) # adding vgg16 model
model.add(Dense(units = 512 , activation = "relu")) # Fully connected layer of 512 units
model.add(Dense(units = 10 , activation = "softmax")) # output layer with 10 units as number of class = 10

model.layers[0].trainable = False

model.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 14,982,474
Trainable params: 267,786
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam' , loss = 'categorical_crossentropy' , metrics = ['acc'])

In [ ]:
# Train the model using the augmented images
model.fit(train_generator,
          steps_per_epoch=len(x_train) // 32,
          epochs=10)

Epoch 1/10
1875/1875 [==============================] - 76s 35ms/step - loss: 0.4837 - acc: 0.8480
Epoch 2/10
1875/1875 [==============================] - 66s 35ms/step - loss: 0.2673 - acc: 0.9138
Epoch 3/10
1875/1875 [==============================] - 65s 34ms/step - loss: 0.2304 - acc: 0.9248
Epoch 4/10
1875/1875 [==============================] - 65s 35ms/step - loss: 0.2124 - acc: 0.9302
Epoch 5/10
1875/1875 [==============================] - 65s 35ms/step - loss: 0.1972 - acc: 0.9358
Epoch 6/10
1875/1875 [==============================] - 65s 35ms/step - loss: 0.1877 - acc: 0.9382
Epoch 7/10
1875/1875 [==============================] - 64s 34ms/step - loss: 0.1758 - acc: 0.9431
Epoch 8/10
1875/1875 [==============================] - 62s 33ms/step - loss: 0.1714 - acc: 0.9430
Epoch 9/10
1875/1875 [==============================] - 62s 33ms/step - loss: 0.1661 - acc: 0.9459
Epoch 10/10
1875/1875 [==============================] - 62s 33ms/step - loss: 0.1596 - acc: 0.9477


In [ ]:
history = model.evaluate(x_test , y_test , verbose = 0)
acc = history[1] * 100

print("Accuracy: %.2f" % acc,"%")

Accuracy: 94.88 %
